# Objektorientierte Programmierung

Ziel: Rechnen mit "Zahlen" auf Restklassenkörpern (Modul-Arithmetik).

Reste bei Addition und Multiplikation modulo n bilden einen Körper, wenn n eine Primzahl ist.

Wir definieren eine Klasse `FiniteFieldElement` (Körper: engl. field), die die Addition und Multiplikation modulo p (prim) implementiert.

In [37]:
class FiniteFieldElement:
    # Konstruktor
    def __init__(self, z, p):
        self.p = p
        self.z = z % p

    def __add__(self, z2):
        if self.p != z2.p:
            raise TypeError("Die Addition ist nicht definiert für Elemente unterschiedlicher Körper")
        return FiniteFieldElement( (self.z + z2.z) % self.p, self.p)

    def __mul__(self, z2):
        if self.p != z2.p:
            raise TypeError("Die Multiplikation ist nicht definiert für Elemente unterschiedlicher Körper")
        return FiniteFieldElement( (self.z * z2.z) % self.p, self.p)
    
    # Darstellung des Objekts als String
    def __str__(self):
        return f"{self.z} mod {self.p}"


In [38]:
z1 = FiniteFieldElement(13, 5)
print(z1)

3 mod 5


In [39]:
z2 = FiniteFieldElement(2, 5)
print(f"{z1} + {z2} = {z1 + z2}")
print(f"{z1} * {z2} = {z1 * z2}")

3 mod 5 + 2 mod 5 = 0 mod 5
3 mod 5 * 2 mod 5 = 1 mod 5
